In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import jax
import jax.numpy as jnp
import numpy as np

from lei_obj import Lewis45

/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
params = {
    "n_arms" : 2,
    "n_stage_1" : 50,
    "n_interims" : 3,
    "n_add_per_interim" : 100,
    "futility_threshold" : 0.1,
    "n_stage_2" : 100,
    "pps_threshold_lower" : 0.1,
    "pps_threshold_upper" : 0.9,
    "posterior_difference_threshold" : 0.1,
    "rejection_threshold" : 0.05,
}

lei_obj = Lewis45(**params)
p = jnp.zeros(2)
grid_points = jnp.array([p] * 1000)
n_sims = 1
key1 = jax.random.split(jax.random.PRNGKey(0), num=4)
keyN = jax.random.split(jax.random.PRNGKey(0), num=n_sims * 4).reshape((n_sims, 4, 2))

In [8]:
print(len(jax.make_jaxpr(lei_obj.single_sim)(p, key1).pretty_print()))
print(len(jax.make_jaxpr(lei_obj.simulate_point)(p, keyN).pretty_print()))
print(len(jax.make_jaxpr(lei_obj.posterior_sigma_sq)(np.random.rand(2,2)).pretty_print()))
# jax.make_jaxpr(lei_obj.posterior_sigma_sq)(np.random.rand(2,2))

235079
247189
12895


In [9]:
%%time
lei_obj.single_sim(p, keyN[0])
# rejections = jax.jit(lei_obj.single_sim)(p, key1)
# jax.vmap(lei_obj.single_sim, in_axes=(None, 0))(p, keyN)
# rejections = jax.jit(lei_obj.simulate_point)(p, keyN)
#rejections = jax.jit(lei_obj.simulate, static_argnums=(0, 3))(n_sims, grid_points, key, 1)

CPU times: user 1.75 s, sys: 23.5 ms, total: 1.77 s
Wall time: 1.87 s


DeviceArray(True, dtype=bool)